mp4 to jpg

In [12]:
import os
import cv2
from utils import createDirectory

# jpg로 변환할 비디오들이 담긴 폴더 지정
video_root_path = '/opt/ml/input/data/videos'

# 저장되는 폴더 경로 지정
image_root_path = '/opt/ml/input/data/images'
createDirectory(image_root_path)

video_list = [os.path.join(video_root_path, i) for i in os.listdir(video_root_path)]
image_list = [os.path.join(image_root_path, i) for i in os.listdir(video_root_path)]

for file_path, save_path in zip(video_list, image_list):
    createDirectory(save_path[:-4])

    start = file_path.find('/', file_path.find('/', file_path.find('/', file_path.find('/', file_path.find('/', file_path.find('/') + 1)+1)+1)+1)+1)

    video_name = file_path[start+1:-4]
    print('convert mp4 to jpg :', video_name)

    cap = cv2.VideoCapture(file_path)
    FPS = cap.get(5) 
    frame = cap.get(7)
    count = 1

    if FPS == 30:
        for i in range(1, int(frame)+1):
            success, img = cap.read()
            
            if not success:
                break

            if i % (1.25) < 1:
                cv2.imwrite(file_path[:-4] + f"/{video_name}"+'-'+str(count).zfill(6)+'.jpg', img)
                count+=1
    elif FPS == 24:
        for i in range(1, int(frame)+1):
            success, img = cap.read()

            if not success:
                break

            height, width, channel = img.shape
            
            # if width > 640:
            #     d = width/640
            #     sc = 1/d
            #     img = cv2.resize(img,dsize=(0,0),fx=sc,fy=sc)

            img = cv2.resize(img, (640, 360))

            cv2.imwrite(save_path[:-4] + f"/{video_name}"+'-'+str(count).zfill(6)+'.jpg', img)
            count+=1
    else:
        print("*"*30)
        print(f"FPS : {FPS} - {file_path}")
        print("*"*30)


convert mp4 to jpg : Cold.Eyes_0-00-04_0-00-13_0-00-20_0-00-31_0-01-02_0-01-04_0-01_21_0-01-24_label_P


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable

import torchvision
from torchvision import datasets, transforms

import videotransforms

import numpy as np
from pytorch_i3d import InceptionI3d
from charades_dataset_full import Charades as Dataset

In [14]:
test_transforms = transforms.Compose([])
first_transforms = transforms.Compose([videotransforms.FirstCrop(224)])
second_transforms = transforms.Compose([videotransforms.SecondCrop(224)])
third_transforms = transforms.Compose([videotransforms.ThirdCrop(224)])
fourth_transforms = transforms.Compose([videotransforms.FourthCrop(224)])
center_transforms = transforms.Compose([videotransforms.CenterCrop(224)])

In [33]:
root = '/opt/ml/input/data/'
mode = 'rgb'
batch_size = 1
save_dir = '/opt/ml/input/data/image_features'
load_model = '/opt/ml/input/code/project/pytorch-i3d/models/rgb_imagenet.pt'

first_dataset = Dataset(root, mode, first_transforms, num=-1, save_dir=save_dir)
first_dataloader = torch.utils.data.DataLoader(first_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

second_dataset = Dataset(root, mode, second_transforms, num=-1, save_dir=save_dir)
second_dataloader = torch.utils.data.DataLoader(second_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

third_dataset = Dataset(root, mode, third_transforms, num=-1, save_dir=save_dir)
third_dataloader = torch.utils.data.DataLoader(third_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

fourth_dataset = Dataset(root, mode, fourth_transforms, num=-1, save_dir=save_dir)
fourth_dataloader = torch.utils.data.DataLoader(fourth_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

center_dataset = Dataset(root, mode, center_transforms, num=-1, save_dir=save_dir)
center_dataloader = torch.utils.data.DataLoader(center_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

# center_dataloader : center|
# first_dataloader : top-left
# second_dataloader : top-right
# third_dataloader : bottom-right
# fourth_dataloader : bottom-left

dataloaders = {
               '0': center_dataloader,
               '1': first_dataloader,
               '2': second_dataloader,
               '4': third_dataloader,
               '3': fourth_dataloader,
                }

datasets = {
            '0': center_dataset,
            '1': first_dataset,
            '2': second_dataset,
            '4': third_dataset,
            '3': fourth_dataset,
            }  

i3d = InceptionI3d(400, in_channels=3)
i3d.replace_logits(400)
i3d.load_state_dict(torch.load(load_model))
i3d.cuda()
i3d.train(False)  # Set model to evaluate mode
        
# Iterate over data.
saved_list = []
for phase in ['0', '1', '2', '3', '4']:
    for data in dataloaders[phase]:
        # get the inputs
        inputs, labels, name = data
        b,c,t,h,w = inputs.shape
        features = []
        for start in range(0, t, 16): 
            end = start + 16 
            ip = Variable(torch.from_numpy(inputs.numpy()[:,:,start:end]).cuda())
            if ip.shape[2] != 16:
                continue
            features.append(i3d.extract_features(ip).squeeze(0).permute(1,2,3,0).data.cpu().numpy())
        np.save(os.path.join(save_dir, name[0]+f"__{phase}"), np.concatenate(features, axis=0).reshape(-1, 1024))
        saved_list.append(f"{os.path.join(save_dir, name[0]+'__'+phase)}.npy")
        print(f"{os.path.join(save_dir, name[0]+'__'+phase)}.npy")    

/opt/ml/input/data/image_features/Cold.Eyes_0-00-04_0-00-13_0-00-20_0-00-31_0-01-02_0-01-04_0-01_21_0-01-24_label_P__0.npy
/opt/ml/input/data/image_features/Cold.Eyes_0-00-04_0-00-13_0-00-20_0-00-31_0-01-02_0-01-04_0-01_21_0-01-24_label_P__1.npy
/opt/ml/input/data/image_features/Cold.Eyes_0-00-04_0-00-13_0-00-20_0-00-31_0-01-02_0-01-04_0-01_21_0-01-24_label_P__2.npy
/opt/ml/input/data/image_features/Cold.Eyes_0-00-04_0-00-13_0-00-20_0-00-31_0-01-02_0-01-04_0-01_21_0-01-24_label_P__3.npy
/opt/ml/input/data/image_features/Cold.Eyes_0-00-04_0-00-13_0-00-20_0-00-31_0-01-02_0-01-04_0-01_21_0-01-24_label_P__4.npy


In [34]:
for sample in saved_list:
    data = np.load(sample)
    print(f'{sample[len(save_dir)+1:]} - {data.shape}')

Cold.Eyes_0-00-04_0-00-13_0-00-20_0-00-31_0-01-02_0-01-04_0-01_21_0-01-24_label_P__0.npy - (139, 1024)
Cold.Eyes_0-00-04_0-00-13_0-00-20_0-00-31_0-01-02_0-01-04_0-01_21_0-01-24_label_P__1.npy - (139, 1024)
Cold.Eyes_0-00-04_0-00-13_0-00-20_0-00-31_0-01-02_0-01-04_0-01_21_0-01-24_label_P__2.npy - (139, 1024)
Cold.Eyes_0-00-04_0-00-13_0-00-20_0-00-31_0-01-02_0-01-04_0-01_21_0-01-24_label_P__3.npy - (139, 1024)
Cold.Eyes_0-00-04_0-00-13_0-00-20_0-00-31_0-01-02_0-01-04_0-01_21_0-01-24_label_P__4.npy - (139, 1024)
